In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [52]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 5

START_DATE = "20210101"
END_DATE = "20241231"

RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
SIDES_DIR = BASE_DIR / "interm/sides"
META_LABELS_DIR = BASE_DIR / "labels/meta_labels"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_FEATURES.pkl"

In [53]:
SIDE_NAME = "EMACross_9_20"
SIDE_FILE_PATH = SIDES_DIR / f"{RESAMPLED_NAME}-{SIDE_NAME}.pkl"

In [54]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)
sides = pd.read_pickle(SIDE_FILE_PATH)

CPU times: user 1.19 ms, sys: 19.6 ms, total: 20.8 ms
Wall time: 44.5 ms


In [55]:
t_events = sides.index
df.loc[t_events].head()

,open,high,low,close,volume,spread,close_pct_return,close_return,close_log_return,close_fd_return,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-07 03:25:00,103.1615,103.1670,103.148,103.1505,7.225200e+11,0.003158,-0.000092,-0.0095,-0.000092,1.913549,...,7,1,7.071068e-01,0.707107,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025
2021-01-07 05:00:00,103.1275,103.1480,103.127,103.1435,1.899900e+11,0.002732,0.000160,0.0165,0.000160,1.931184,...,7,1,9.659258e-01,0.258819,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025
2021-01-07 12:45:00,103.5910,103.6025,103.571,103.5905,8.012900e+11,0.002160,-0.000014,-0.0015,-0.000014,1.925384,...,7,1,1.224647e-16,-1.000000,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025
2021-01-07 14:20:00,103.6250,103.6510,103.625,103.6465,1.681110e+12,0.002214,0.000203,0.0210,0.000203,1.983158,...,7,1,-5.000000e-01,-0.866025,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025
2021-01-07 16:35:00,103.7840,103.8030,103.767,103.7940,1.122950e+12,0.002712,0.000111,0.0115,0.000111,1.950915,...,7,1,-8.660254e-01,-0.500000,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025


## Calculate Target Volatility (trgt)
This will be used for the calculation of dynamic TP/SL

In [56]:
import pandas_ta as ta

In [57]:
df.ta.atr(length=9, append=True)
df['ATRr_9_PERC'] = df.ta.atr(length=0, percent=True)
df.dropna(inplace=True)
df.head()

,open,high,low,close,volume,spread,close_pct_return,close_return,close_log_return,close_fd_return,...,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos,ATRr_9,ATRr_9_PERC
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-07 04:35:00,103.1195,103.1315,103.1105,103.1210,7.896400e+11,0.002230,-0.000015,-0.0015,-0.000015,1.913404,...,0.866025,0.5,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025,0.020080,0.019325
2021-01-07 04:40:00,103.1200,103.1300,103.1185,103.1275,2.914800e+11,0.002024,0.000063,0.0065,0.000063,1.920634,...,0.866025,0.5,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025,0.019127,0.018740
2021-01-07 04:45:00,103.1270,103.1330,103.1265,103.1330,1.739200e+11,0.002074,0.000053,0.0055,0.000053,1.923142,...,0.866025,0.5,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025,0.017724,0.017851
2021-01-07 04:50:00,103.1330,103.1390,103.1320,103.1380,1.364700e+11,0.002395,0.000048,0.0050,0.000048,1.924756,...,0.866025,0.5,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025,0.016532,0.017060
2021-01-07 04:55:00,103.1355,103.1385,103.1265,103.1270,2.892900e+11,0.002637,-0.000107,-0.0110,-0.000107,1.910298,...,0.866025,0.5,0.433884,-0.900969,0.988468,0.151428,0.5,0.866025,0.016029,0.016674


In [58]:
trgt = df['ATRr_9_PERC'].reindex(t_events) / 100

In [59]:
trgt.describe()

count    13977.000000
mean         0.000382
std          0.000240
min          0.000045
25%          0.000222
50%          0.000324
75%          0.000480
max          0.003381
Name: ATRr_9_PERC, dtype: float64

## Apply Triple Barrier
We use daily volatility to calculate TP/SL, and we set our T1 as 1day

In [60]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd

def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    
    pt = ptSl[0] * events_['trgt'] if ptSl[0] > 0 else pd.Series(index=events.index)
    sl = -ptSl[1] * events_['trgt'] if ptSl[1] > 0 else pd.Series(index=events.index)

        
    for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
        df0=close[loc:t1] # path prices, i.e. price[t0:t1]
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
    return out


def parallel_apply(func, items, num_threads=4, **kwargs):
    
    def worker(molecule):
        return func(molecule=molecule, **kwargs)

    chunks = np.array_split(items, num_threads)
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(worker, chunks))

    return pd.concat(results).sort_index()


def getEvents(close, tEvents, ptSl, trgt, minTrgt, numThreads=4, t1=False, side=None):
    # Step 1: Filter targets
    trgt = trgt.loc[tEvents]
    trgt = trgt[trgt > minTrgt]

    # Step 2: Set vertical barrier (t1)
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=tEvents)

    # Step 3: Build events DataFrame
    if side is None:
        side_, ptSl_ = pd.Series(1., index=trgt.index), [ptSl[0], ptSl[0]]
    else:
        side_, ptSl_ = side.loc[trgt.index], ptSl[:2]

    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1).dropna(subset=['trgt'])

    # Step 4: Apply barriers in parallel
    df0 = parallel_apply(
        func=applyPtSlOnT1,
        items=events.index,
        num_threads=numThreads,
        close=close,
        events=events,
        ptSl=ptSl_
    )

    # Step 5: Choose the first touched barrier
    events['t1'] = df0.dropna(how='all').min(axis=1)
    
    if side is None:
        events = events.drop('side', axis=1)
        
    return events


In [61]:
def get_vertical_barrier(t_events, close, delta=pd.Timedelta(minutes=60)):
    """
    For each event in t_events, find the timestamp in close.index 
    that is at least delta later. Returns a pd.Series of t1.
    """
    barrier_times = t_events + delta
    t1_idx = close.index.searchsorted(barrier_times)
    valid_idx = t1_idx[t1_idx < len(close)]
    t1 = pd.Series(close.index[valid_idx], index=t_events[:len(valid_idx)])
    return t1

In [62]:
t1 = get_vertical_barrier(t_events, df['close'], delta=pd.Timedelta(minutes=60))

In [63]:
t1

timestamp
2021-01-07 03:25:00   2021-01-07 04:35:00
2021-01-07 05:00:00   2021-01-07 06:00:00
2021-01-07 12:45:00   2021-01-07 13:45:00
2021-01-07 14:20:00   2021-01-07 15:20:00
2021-01-07 16:35:00   2021-01-07 17:35:00
                              ...        
2024-12-30 08:05:00   2024-12-30 09:05:00
2024-12-30 09:50:00   2024-12-30 10:50:00
2024-12-30 16:30:00   2024-12-30 17:30:00
2024-12-30 18:00:00   2024-12-30 19:00:00
2024-12-30 22:25:00   2024-12-30 23:25:00
Name: timestamp, Length: 13978, dtype: datetime64[ns]

In [64]:
trgt.describe()

count    13977.000000
mean         0.000382
std          0.000240
min          0.000045
25%          0.000222
50%          0.000324
75%          0.000480
max          0.003381
Name: ATRr_9_PERC, dtype: float64

In [65]:
minTrgt = trgt.quantile(.3)
minTrgt

np.float64(0.0002403797202206965)

In [66]:
%%time

events = getEvents(
    close=df['close'],
    tEvents=t_events, 
    ptSl=[.5,.5], 
    trgt=trgt, 
    minTrgt=minTrgt, 
    numThreads=20, 
    t1=t1, 
    side=sides
)

CPU times: user 2.35 s, sys: 53.3 ms, total: 2.4 s
Wall time: 2.37 s


In [67]:
events.head()

,t1,trgt,side
timestamp,,,
2021-01-07 12:45:00,2021-01-07 12:50:00,0.000409,-1.0
2021-01-07 14:20:00,2021-01-07 14:25:00,0.000399,1.0
2021-01-07 16:35:00,2021-01-07 16:55:00,0.000527,-1.0
2021-01-07 17:00:00,2021-01-07 17:10:00,0.000485,1.0
2021-01-07 18:25:00,2021-01-07 18:30:00,0.000318,-1.0


In [68]:
events = events.dropna().copy()

# Keep only intraday events

In [69]:
events = events[events.index.date==events.t1.dt.date]

## Create classification labels

In [70]:
def getBins(events, close, minRet=0.0, t1=None):
    '''
    Compute event's outcome (including side information, if provided).
    events is a DataFrame where:
    -events.index is event's starttime
    -events['t1'] is event's endtime
    -events['trgt'] is event's target
    -events['side'] (optional) implies the algo's position side
    -t1 is original vertical barrier series
    Case 1: ('side' not in events): bin in (-1,1) <-label by price action
    Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    '''
    # 1) prices aligned with events
    events_ = events.dropna(subset=['t1'])
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # 2) create out object
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[
        events_.index] - 1
    if 'side' in events_: out['ret'] *= events_['side']  # meta-labeling
    out['bin'] = np.sign(out['ret'])

    if 'side' not in events_:
        # only applies when not meta-labeling.
        # to update bin to 0 when vertical barrier is touched, we need the
        # original vertical barrier series since the events['t1'] is the time
        # of first touch of any barrier and not the vertical barrier
        # specifically. The index of the intersection of the vertical barrier
        # values and the events['t1'] values indicate which bin labels needs
        # to be turned to 0.
        vtouch_first_idx = events[events['t1'].isin(t1.values)].index
        out.loc[vtouch_first_idx, 'bin'] = 0.

    if 'side' in events_: out.loc[out['ret'] <= 0, 'bin'] = 0  # meta-labeling
    # out.loc[out['ret'].abs() < minRet, 'bin'] = 0. # 不做獲利過少的交易 (設為 0)
    # out = out.loc[out['ret'].abs() >= minRet] # 過濾掉獲利過少的交易 (刪掉)
    return out

In [71]:
labels = getBins(events, df['close'], minRet=0, t1=t1)

In [72]:
labels.bin.value_counts()

bin
0.0    5039
1.0    4697
Name: count, dtype: int64

In [73]:
labels = labels.join(events)

In [74]:
LABELS_FILE_PATH = META_LABELS_DIR / f"{RESAMPLED_NAME}-{SIDE_NAME}-meta.pkl"

In [75]:
labels.to_pickle(LABELS_FILE_PATH)